# เตรียมลิสต์ไฟล์เรดาร์กับฝนสถานีเพื่อนำไปหาค่า MFB แต่ละสถานี
* เป็นการจับคู่ชื่อไฟล์เรดาร์กับชื่อไฟล์ฝนสถานีแบบ local time เพื่อนำไปใช้ในการหาค่า MFB

In [1]:
'''
2024.09.22
โค้ดนี้พัฒนาโดย รองศาสตราจารย์ ดร. นัฐพล มหาวิค ภาควิชาทรัพยากรธรรมชาติและสิ่งแวดล้อม คณะเกษตรศาสตร์ฯ มหาวิทยาลัยนเรศวร 
ในงานวิจัย เรื่อง "การวิจัยและพัฒนาผลิตภัณฑ์โมเสคฝนประมาณค่าจากเรดาร์ตรวจอากาศในพื้นที่ระดับลุ่มน้ำของประเทศไทยด้วยเทคโนโลยีภูมิสารสนเทศรหัสเปิด"
สนับสนุนทุนวิจัยโดยสํานักงานการวิจัยแห่งชาติ (วช.)  แผนงานการวิจัยและนวัตกรรมแผนงานด้านการบริหารจัดการภัยพิบัติทางธรรมชาติ 
ประจำปีงบประมาณ 2566  ตามสัญญา เลขที่ N25A660467 ผู้นำโค้ดนี้ไปใช้หรือดัดแปลงควรอ้างอิงงานวิจัยชิ้นนี้ตามหลักเกณฑ์การอ้างอิงสากล
เรียนหลักการเรดาร์และภูมิสารสนเทศ ที่ https://www.youtube.com/@Nattapon_Mahavik/playlists
หนังสือเรดาร์ตรวจอากาศทางอุตุนิยมวิทยา สำนักพิมพ์จุฬาฯ : https://www.chulabook.com/education/144567
หนังสือออนไลน์เรดาร์ตรวจอากาศทางอุตุนิยมวิทยา สำนักพิมพ์จุฬาฯ : https://www.chulabook.com/education/205129
ติดต่อ nattaponm@nu.ac.th
'''
import os
import datetime
import pandas as pd
from pathlib import Path

def get_file_datetime(filename, is_radar=True):
    # Extract datetime from filename
    date_str = filename.split('.')[0]  # Remove the extension
    return datetime.datetime.strptime(date_str, '%Y%m%d%H')

def get_matching_files(radar_dir, gauge_dir):
    radar_files = os.listdir(radar_dir)
    gauge_files = os.listdir(gauge_dir)
    
    radar_times = {get_file_datetime(f, is_radar=True): f for f in radar_files if f.endswith('.tif')}
    gauge_times = {get_file_datetime(f, is_radar=False): f for f in gauge_files if f.endswith('.csv')}
    
    matched_times = sorted(set(radar_times.keys()) & set(gauge_times.keys()))
    
    return [(t, radar_times[t], gauge_times[t]) for t in matched_times]

def save_matched_list(matched_list, output_file):
    df = pd.DataFrame(matched_list, columns=['Datetime', 'Radar_File', 'Gauge_File'])
    df.to_csv(output_file, index=False)
    print(f"Matched list saved to {output_file}")

def process_radar_station(radar_name, radar_dir, gauge_dir, output_dir):
    radar_station_dir = os.path.join(radar_dir, f"00outp_cappi_2km_{radar_name}")
    gauge_station_dir = os.path.join(gauge_dir, radar_name, 'hourly')
    
    if not os.path.exists(radar_station_dir):
        print(f"Radar directory for {radar_name} not found: {radar_station_dir}")
        return
    if not os.path.exists(gauge_station_dir):
        print(f"Gauge directory for {radar_name} not found: {gauge_station_dir}")
        return
    
    matched_list = get_matching_files(radar_station_dir, gauge_station_dir)
    
    output_file = os.path.join(output_dir, f"matched_files_{radar_name}.csv")
    save_matched_list(matched_list, output_file)
    
    print(f"Processed {radar_name}: {len(matched_list)} matches found.")

def main():
    radar_dir = '../00run_batch_acchr_codes/2output/0Hourly/0Sontihn_dbz_single/0processing_single_cappi/' #ปรับเปลี่ยนได้ที่เก็บข้อมูลเรดาร์
    gauge_dir = './0Zprocessing_data/0hourly_rain_each_radar/' #ปรับเปลี่ยนได้ที่เก็บข้อมูลฝนรายชั่วโมง
    output_dir = './0Zprocessing_data/0match_files/' #ที่เก็บผลลัพธ์การจับคู่ไฟล์
    
    # Ensure output directory exists
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # List of radar names
    radar_names = ['CHN', 'CMP', 'CRI', 'KKN', 'KRB', 'LMP', 'NAN', 'NRT', 'PHS', 'PKT', 'SNK', 'STP', 'SVP', 'URT']
    
    for radar in radar_names:
        process_radar_station(radar, radar_dir, gauge_dir, output_dir)
    
    print("Processing complete. Check the output directory for matched files.")

if __name__ == "__main__":
    main()

Matched list saved to ./0Zprocessing_data/0match_files/matched_files_CHN.csv
Processed CHN: 240 matches found.
Matched list saved to ./0Zprocessing_data/0match_files/matched_files_CMP.csv
Processed CMP: 240 matches found.
Matched list saved to ./0Zprocessing_data/0match_files/matched_files_CRI.csv
Processed CRI: 240 matches found.
Matched list saved to ./0Zprocessing_data/0match_files/matched_files_KKN.csv
Processed KKN: 240 matches found.
Matched list saved to ./0Zprocessing_data/0match_files/matched_files_KRB.csv
Processed KRB: 227 matches found.
Matched list saved to ./0Zprocessing_data/0match_files/matched_files_LMP.csv
Processed LMP: 240 matches found.
Matched list saved to ./0Zprocessing_data/0match_files/matched_files_NRT.csv
Processed NRT: 240 matches found.
Matched list saved to ./0Zprocessing_data/0match_files/matched_files_PHS.csv
Processed PHS: 237 matches found.
Matched list saved to ./0Zprocessing_data/0match_files/matched_files_PKT.csv
Processed PKT: 215 matches found.
M